In [1]:
import matplotlib.pyplot as plt
import numpy as np
from KDEpy import NaiveKDE
import scipy.integrate as integrate
from function_library_3 import trim
from tqdm import tqdm

#================================================================================
# SETUP

#change default font size for graphs
plt.rcParams.update({'font.size':20})

# create the needed directories and clear old output
import os,glob
try:
    os.mkdir('3_Pressure_Distributions')
except:
    for filename in glob.glob('3_Pressure_Distributions/*'):
        os.remove(filename)
try:
    os.mkdir('3_Temperature_Distributions')
except:
    for filename in glob.glob('3_Temperarture_Distributions/*'):
        os.remove(filename)


# suppress warnings
import warnings
warnings.filterwarnings('ignore')


#================================================================================
# RETRIEVE BASIN SIZE DATA

# open the basin input file
input_file = open('Basins.txt','r')

# create a hashmap from basin name -> diameter
basin_diameter = {} 
line = ' '
while line[0:3] != 'END':
    line = input_file.readline()
    if line[0:2] == 'LB': # data is retrieved from lines starting with LB: (Lunar Basin)
        name, size, N, E = trim(line.split(':'))
        basin_diameter[name] = size
        
input_file.close()
    
#================================================================================
# DEFINE HAVERSINE FORMULA
    
def haversine(lat1, lon1, lat2, lon2, Radius): # haversine formula
    dLat = np.radians(lat2 - lat1) # change in latitude
    dLon = np.radians(lon2 - lon1) # change in longitude
    lat1 = np.radians(lat1) # latitude 1
    lat2 = np.radians(lat2) # latitude 2
    a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2 # intermediate step
    c = 2*np.arcsin(np.sqrt(a)) # interior angle between two coordinates [rad]
    return Radius * c # returns distance between two coordinates

#================================================================================
# GLOBAL PARAMETERS

p_Range = [.1,310] # plotted pressure range
T_Range = [200,14000] # plotted temperature range
    
basin_list = [] # generate list of all basins considered
for filename in os.listdir('1_Basin_Data'):
    if filename.endswith(".dat"):
        basin_list.append(filename)

# hashmaps of Lunar missions and their coordinates
A_sites = {11:[0.67409,23.47298],12:[-3.01381,-23.41930],14:[-3.64544,-17.47139],15:[26.13224,3.63400],16:[-8.97341,15.49859],17:[20.18809,30.77475]}
L_sites = {16:[-.5137,56.3638],20:[3.5333,56.55],24:[12.7145,62.2129]}
C_sites = {5:[43.1,-51.8]}

#================================================================================
# DEFINE CROSS-SECTION FUNCTION

def cross_section(loc, param):  # take a slice of the pressure and temperature 2D KDEs  
    # basin name    2D KDE values    Stacked Height values   Loc of basin, Radius planet, Radius basin 
    basin,         x, yp,zp, yT,zT,   height_x, height_y,                   coords, R_p, R_c           = param
    #             loc  pres   Temp     loc       height                      N,E
    
    #---------------------------------------
    # data is interpolated linearly between the two surounding columns closest the to landing location
    
    lower = min(x) ; upper = max(x) # find the lower and upper bounds of the landing location data
    
    # find the below and above cross-section
    for i in range(len(x)):
        if loc < x[i] <= upper:
            upper = x[i]
        if lower < x[i] <= loc:
            lower = x[i]
    # if the location is out of bounds, return null values       
    if lower == upper or max(height_x) < loc:
        return 0,[],[],[],[]
    
    #---------------------------------------
    # interpolate stacked height curve 
    i=0 ; height_accum = 0
    while i+1 < len(height_x):
        if height_x[i] <= loc <= height_x[i+1]: # locate the two points that need to be interpolated
            xl=height_x[i] ; xh=height_x[i+1] ; c=loc # x values of data below and above the loc
            yl=height_y[i] ; yh=height_y[i+1] # y values of data below and above
            height_accum = yl+(yh-yl)/(xh-xl)*(c-xl) # linearly interpolate
            break
        i+=1  
    
    #---------------------------------------
    # initialize lists of the cross-sections below and above the landing location
    pres0,pres1 = [],[] ; Temp0,Temp1 = [],[] # pressures, probabilty density ; pressures, probabilty density 
    pval0,pval1 = [],[] ; Tval0,Tval1 = [],[] # temperature, probabilty density ; temperature, probabilty density 

    # create the lists of the cross-sections above and below the landing location 
    for i in range(len(x)):
        if x[i] == lower: # below cross-sections
            pres0.append(yp[i])
            pval0.append(zp[i])
            Temp0.append(yT[i])
            Tval0.append(zT[i])
        if x[i] == upper: # above cross-sections
            pres1.append(yp[i])
            pval1.append(zp[i])
            Temp1.append(yT[i])
            Tval1.append(zT[i])
    
    # intialize lists of the cross-section at the landing location
    pres,pval=[],[] # pressures, probabilty density
    Temp,Tval=[],[] # temperature, probabilty density
    
    # linearly interpolate the below and above cross-section curves to get the desired cross-section      
    for i in range(len(pres0)):
        pres.append(pres0[i]) # append the pressures 
        Temp.append(Temp0[i]) # append the temperatures 
        
        xl=lower ; xh=upper ; c=loc # locate the two cross-sections that need to be interpolated
        
        # interpolate the pressure cross-sections
        yl=pval0[i] ; yh=pval1[i]
        value = yl+(yh-yl)/(xh-xl)*(c-xl) # linearly interpolate
        pval.append(value)
        
        # interpolate the temperature cross-sections
        yl=Tval0[i] ; yh=Tval1[i]
        value = yl+(yh-yl)/(xh-xl)*(c-xl) # linearly interpolate
        Tval.append(value)
        
    #---------------------------------------
    # returns a stacked height value, pressure curve, temperature curve
    return height_accum, np.array(pres), np.array(pval), np.array(Temp), np.array(Tval)

#================================================================================
# DEFINE MAIN FUNCTION

def distribution(Apollo=False,Luna=False,Chang_e=False): # create pressure distributions at the sample site
    
    # error catch if more than one mission is specified
    if bool(Apollo) + bool(Luna) + bool(Chang_e) > 1:
        print('ERROR : Can only take 1 keyword argument')
        return
    
    #---------------------------------------
    # get user input for the sample location (or draw from mission hashmap)
    if Apollo: # get user input if Apollo
        if Apollo == True:
            mission = eval(input('Apollo Mission : '))
        else:
            mission = Apollo
        N_sample = A_sites[mission][0]
        E_sample = A_sites[mission][1]
        title_loc = 'Apollo {} Landing Site'.format(mission)
    elif Luna: # get user input if Luna
        if Luna == True:
            mission = eval(input('Luna Mission : '))
        else:
            mission = Luna
        N_sample = L_sites[mission][0]
        E_sample = L_sites[mission][1]
        title_loc = 'Luna {} Landing Site'.format(mission)
    elif Chang_e: # get user input if Chang'e
        if Chang_e == True:
            mission = eval(input('Chang_e Mission : '))
        else:
            mission = Chang_e
        N_sample = C_sites[mission][0]
        E_sample = C_sites[mission][1]
        title_loc = "Chang'e {} Landing Site".format(mission)
        
    else: # get user input of specified coordinates
        N_sample = eval(input('Sample Site location Degrees North: '))
        E_sample = eval(input('Sample Site location Degrees East : '))
        title_loc = '{} N , {} E'.format(N_sample,E_sample)
    
    print('============================================================================')
    if Apollo or Luna or Chang_e:
        print(title_loc)
    else:
        print('Custom Site')
    print('Sample Location: {} N , {} E'.format(N_sample,E_sample))
    print('============================================================================')
    
    #---------------------------------------
    # begin combining the pressure/temperature distributions
    
    # initialize lists 
    names = [] ; radii = [] ; distances = [] # names of each basin, diameter of basin, and distance from sample site to center of basin
    heights, px, pv, Tx, Tv = [],[],[],[],[] 
    # respective stacked heights and pressure/temperature distributions from each basin
    
    # cycle through each basin for their contribution
    for each in tqdm(basin_list):
        # extract the data from the KDE data
        fileKDE  = open('1_KDE_Data/{}'.format(each),'r')
        full_data = eval(fileKDE.readline())
        fileKDE.close()
        names.append(full_data[0]) # names of each basin
        N_basin = full_data[8][0] # latitude of basin
        E_basin = full_data[8][1] # longitude of basin
        planet_radius = full_data[9] # radius of planet
        distances.append(haversine(N_basin,E_basin,N_sample,E_sample,planet_radius)) # distance from sample site to center of basin 
        radii.append(full_data[10]) # radius of each basin
        H_0,p_0,pv_0,T_0,Tv_0 = cross_section(distances[-1],full_data) # respective stacked heights and pressure/temperature distributions from each basin
        # add to each list
        heights.append(H_0)
        px.append(p_0) ; pv.append(pv_0)
        Tx.append(T_0) ; Tv.append(Tv_0)
        
    #---------------------------------------
    # synchronize the patterns and colors
    
    import random
    random.seed(7)
    lines = ["--","-.",":"]
    pattern_color = {}
    
    for each in names:
        color="#"+''.join([random.choice('0123456789ABCDEF') for i in range(6)])
        line_type = lines[random.randint(0,2)]
        pattern_color[each] = [line_type,color]
        
    #---------------------------------------
    # sort from nearest to farthest basin (sample to center)
    distances, radii, heights, names , px, pv, Tx, Tv = zip(*sorted(zip(distances, radii, heights, names, px, pv, Tx, Tv)))
    
    # all the px and Tx are identical... so we can collapse that array:
    px = px[0]
    Tx = Tx[0]
    
    # sort the stacked-heights so that they can be ranked
    sorted_heights = []
    for each in heights:
        sorted_heights.append(each)
    sorted_heights.sort() ; sorted_heights.reverse()
    
    # rank the stacked-heights
    i = 0 ; rankings = {}
    for each in sorted_heights:
        if each in rankings.keys():
            rankings[each] = i
        else:
            i += 1
            rankings[each] = i        

    print('Stacked Height at {}'.format(title_loc))
    print('{:0.05f} km of ejecta from all basins'.format(sum(heights)))
    print('----------------------------------------------------------------------------')
    for i in range(len(names)):
        if heights[i] < 1e-9 : # skip basin if it has no contributions
            continue
            
        sample_to_center = '| {:04d} km away '.format(int(distances[i]))
        
        sample_to_edge = '| {:04d} km from edge '.format(int(distances[i] - radii[i]))
        if radii[i] > distances[i]: # determine if the sample site is from within the basin
            sample_to_edge = '| {:04d} km within edge '.format(int(radii[i] - distances[i]))
        
        # print the details from each basin
        rank = rankings[heights[i]]
        print('{:0.05f} km (#{}) | from {:12s}  '.format(heights[i],rank,names[i]) + sample_to_center + sample_to_edge)

    print('============================================================================')
    
    # initialize lists
    p_AbunEach, p_AbunTotal = [], 0
    T_AbunEach, T_AbunTotal = [], 0
    for i in range(len(names)): # add up contributions from each basin
        if heights[i] < 1e-9 : # skip basin if it has no contributions
            p_AbunEach.append([]) # add in dummy placeholders for empty non-contributing basins
            T_AbunEach.append([]) # add in dummy placeholders for empty non-contributing basins
            continue
            
        p_norm = integrate.trapz(pv[i], x=px)
        T_norm = integrate.trapz(Tv[i], x=Tx)
        p_AbunEach.append(heights[i]/p_norm*np.array(pv[i])) # normalize by the stacked height / area under the pressure curve
        T_AbunEach.append(heights[i]/T_norm*np.array(Tv[i])) # normalize by the stacked height / area under the temperature curve 
        p_AbunTotal += p_AbunEach[-1] # combined pressure distribution curve
        T_AbunTotal += T_AbunEach[-1] # combined temperature distribution curve
        
    AreaP = integrate.trapz(p_AbunTotal,x=px) # area under the total pressure curve
    AreaT = integrate.trapz(T_AbunTotal,x=Tx) # area under the total temperature curve
    
    
    #---------------------------------------
    # plot the pressure distributions
    
    fig = plt.figure(figsize=(16,16),facecolor='white')
    plt.suptitle('Pressure Distribution at {}'.format(title_loc), fontsize=32)
    ax = fig.add_subplot(111) 
    
    ax.set_title('Pressure PDF', fontsize=24)
    ax.set_xlabel('Pressure [GPa]', fontsize=24)
    ax.set_ylabel('Probability Density (normed to Combined)', fontsize=24)
    
    ax.plot(px, p_AbunTotal/AreaP, c='black', label='(Combined)', alpha=.5) # plot combined distribution
        
    for i in range(len(names)): # plot each basin's pressure distribution
        if heights[i] < 1e-9 : # skip basin if it has no contributions
            continue
        lsc = pattern_color[names[i]][0]
        col = pattern_color[names[i]][1]
        ax.plot(px, p_AbunEach[i]/AreaP, ls=lsc, c=col, label=names[i])
        
    ax.set_xlim(0,60)
    
    #ax.set_yscale('log')
    
    ax.grid(True, ls='--', zorder=-15); ax.legend(fontsize=24,title='In Order of Proximity');
    
    fig.savefig('3_Pressure_Distributions/Pressure Distribution at {}.png'.format(title_loc))
    plt.close(fig)

    #---------------------------------------
    # plot the temperature distributions
    
    fig = plt.figure(figsize=(16,16),facecolor='white')
    plt.suptitle('Temperature Distribution at {}'.format(title_loc), fontsize=32)
    ax = fig.add_subplot(111) 
    
    ax.set_title('Temperature PDF', fontsize=24)
    ax.set_xlabel('Temperature [K]', fontsize=24)
    ax.set_ylabel('Probability Density (normed to Combined)', fontsize=24)
    
    ax.plot(Tx, T_AbunTotal/AreaT, c='black', label='(Combined)', alpha=.5) # plot combined distribution
    
    for i in range(len(names)): # plot each basin's temperature distribution
        if heights[i] < 1e-9 : # skip basin if it has no contributions
            continue
        lsc = pattern_color[names[i]][0]
        col = pattern_color[names[i]][1]
        ax.plot(Tx, T_AbunEach[i]/AreaT, ls=lsc, c=col, label=names[i])
    
    ax.grid(True, ls='--', zorder=-15); ax.legend(fontsize=24,title='In Order of Proximity');
    
    ax.set_xlim(250,3000)
    
    #ax.set_yscale('log')
    
    fig.savefig('3_Temperature_Distributions/Temperature Distribution at {}.png'.format(title_loc))
    plt.close(fig)
    
    

In [2]:
distribution(Apollo=11)

Apollo 11 Landing Site
Sample Location: 0.67409 N , 23.47298 E


100%|███████████████████████████████████████████| 19/19 [02:20<00:00,  7.42s/it]


Stacked Height at Apollo 11 Landing Site
0.01891 km of ejecta from all basins
----------------------------------------------------------------------------
0.00041 km (#4) | from Nectaris      | 0589 km away | 0419 km from edge 
0.00492 km (#2) | from Serenitatis   | 0852 km away | 0515 km from edge 
0.00045 km (#3) | from Crisium       | 1181 km away | 0903 km from edge 
0.01311 km (#1) | from Imbrium       | 1490 km away | 0917 km from edge 
0.00003 km (#5) | from Humorum       | 1987 km away | 1777 km from edge 
0.00000 km (#6) | from SPoleAitkens  | 3676 km away | 2426 km from edge 


In [3]:
distribution(Apollo=12)

Apollo 12 Landing Site
Sample Location: -3.01381 N , -23.4193 E


100%|███████████████████████████████████████████| 19/19 [02:11<00:00,  6.94s/it]


Stacked Height at Apollo 12 Landing Site
0.02948 km of ejecta from all basins
----------------------------------------------------------------------------
0.00047 km (#3) | from Humorum       | 0791 km away | 0581 km from edge 
0.02828 km (#1) | from Imbrium       | 1117 km away | 0544 km from edge 
0.00056 km (#2) | from Serenitatis   | 1529 km away | 1192 km from edge 
0.00000 km (#7) | from Nectaris      | 1783 km away | 1613 km from edge 
0.00000 km (#6) | from Lorentz       | 2336 km away | 2147 km from edge 
0.00005 km (#5) | from Crisium       | 2557 km away | 2279 km from edge 
0.00011 km (#4) | from Hertzsprung   | 3215 km away | 2930 km from edge 


In [4]:
distribution(Apollo=14)

Apollo 14 Landing Site
Sample Location: -3.64544 N , -17.47139 E


100%|███████████████████████████████████████████| 19/19 [02:11<00:00,  6.91s/it]


Stacked Height at Apollo 14 Landing Site
0.02969 km of ejecta from all basins
----------------------------------------------------------------------------
0.00036 km (#3) | from Humorum       | 0887 km away | 0677 km from edge 
0.02846 km (#1) | from Imbrium       | 1114 km away | 0541 km from edge 
0.00072 km (#2) | from Serenitatis   | 1410 km away | 1073 km from edge 
0.00005 km (#5) | from Nectaris      | 1603 km away | 1433 km from edge 
0.00008 km (#4) | from Crisium       | 2391 km away | 2113 km from edge 
0.00000 km (#7) | from Lorentz       | 2497 km away | 2308 km from edge 
0.00003 km (#6) | from Hertzsprung   | 3397 km away | 3112 km from edge 


In [5]:
distribution(Apollo=15)

Apollo 15 Landing Site
Sample Location: 26.13224 N , 3.634 E


100%|███████████████████████████████████████████| 19/19 [02:10<00:00,  6.86s/it]


Stacked Height at Apollo 15 Landing Site
0.39440 km of ejecta from all basins
----------------------------------------------------------------------------
0.08122 km (#2) | from Serenitatis   | 0381 km away | 0044 km from edge 
0.31297 km (#1) | from Imbrium       | 0549 km away | 0023 km within edge 
0.00005 km (#4) | from Nectaris      | 1562 km away | 1392 km from edge 
0.00014 km (#3) | from Crisium       | 1588 km away | 1310 km from edge 
0.00003 km (#5) | from Humorum       | 1983 km away | 1773 km from edge 


In [6]:
distribution(Apollo=16)

Apollo 16 Landing Site
Sample Location: -8.97341 N , 15.49859 E


100%|███████████████████████████████████████████| 19/19 [02:09<00:00,  6.83s/it]


Stacked Height at Apollo 16 Landing Site
0.01380 km of ejecta from all basins
----------------------------------------------------------------------------
0.00039 km (#3) | from Nectaris      | 0600 km away | 0430 km from edge 
0.00169 km (#2) | from Serenitatis   | 1130 km away | 0793 km from edge 
0.00016 km (#4) | from Crisium       | 1537 km away | 1259 km from edge 
0.01142 km (#1) | from Imbrium       | 1564 km away | 0991 km from edge 
0.00008 km (#5) | from Humorum       | 1640 km away | 1430 km from edge 
0.00005 km (#6) | from SPoleAitkens  | 3483 km away | 2233 km from edge 


In [7]:
distribution(Apollo=17)

Apollo 17 Landing Site
Sample Location: 20.18809 N , 30.77475 E


100%|██████████████████████████████████████████| 19/19 [39:05<00:00, 123.44s/it]


Stacked Height at Apollo 17 Landing Site
0.07137 km of ejecta from all basins
----------------------------------------------------------------------------
0.05064 km (#1) | from Serenitatis   | 0439 km away | 0102 km from edge 
0.00171 km (#3) | from Crisium       | 0824 km away | 0546 km from edge 
0.00006 km (#4) | from Nectaris      | 1086 km away | 0916 km from edge 
0.01895 km (#2) | from Imbrium       | 1313 km away | 0741 km from edge 
0.00000 km (#5) | from Harkhebi      | 1839 km away | 1671 km from edge 
0.00000 km (#6) | from Humorum       | 2470 km away | 2260 km from edge 


In [8]:
distribution(Luna=16)

Luna 16 Landing Site
Sample Location: -0.5137 N , 56.3638 E


100%|███████████████████████████████████████████| 19/19 [02:08<00:00,  6.77s/it]


Stacked Height at Luna 16 Landing Site
0.03201 km of ejecta from all basins
----------------------------------------------------------------------------
0.00803 km (#2) | from Crisium       | 0541 km away | 0263 km from edge 
0.00011 km (#5) | from Nectaris      | 0795 km away | 0625 km from edge 
0.00069 km (#4) | from Serenitatis   | 1432 km away | 1095 km from edge 
0.00002 km (#6) | from Harkhebi      | 1692 km away | 1524 km from edge 
0.00247 km (#3) | from Imbrium       | 2296 km away | 1724 km from edge 
0.02069 km (#1) | from SPoleAitkens  | 3145 km away | 1895 km from edge 


In [9]:
distribution(Luna=20)

Luna 20 Landing Site
Sample Location: 3.5333 N , 56.55 E


100%|███████████████████████████████████████████| 19/19 [02:08<00:00,  6.79s/it]


Stacked Height at Luna 20 Landing Site
0.03218 km of ejecta from all basins
----------------------------------------------------------------------------
0.02046 km (#1) | from Crisium       | 0418 km away | 0140 km from edge 
0.00009 km (#5) | from Nectaris      | 0876 km away | 0706 km from edge 
0.00086 km (#4) | from Serenitatis   | 1358 km away | 1021 km from edge 
0.00005 km (#6) | from Harkhebi      | 1594 km away | 1425 km from edge 
0.00282 km (#3) | from Imbrium       | 2232 km away | 1660 km from edge 
0.00791 km (#2) | from SPoleAitkens  | 3242 km away | 1992 km from edge 


In [10]:
distribution(Luna=24)

Luna 24 Landing Site
Sample Location: 12.7145 N , 62.2129 E


100%|███████████████████████████████████████████| 19/19 [02:09<00:00,  6.83s/it]


Stacked Height at Luna 24 Landing Site
0.15142 km of ejecta from all basins
----------------------------------------------------------------------------
0.14661 km (#1) | from Crisium       | 0159 km away | 0118 km within edge 
0.00006 km (#5) | from Nectaris      | 1193 km away | 1023 km from edge 
0.00005 km (#6) | from Harkhebi      | 1270 km away | 1102 km from edge 
0.00087 km (#4) | from Serenitatis   | 1353 km away | 1016 km from edge 
0.00283 km (#2) | from Imbrium       | 2229 km away | 1657 km from edge 
0.00100 km (#3) | from SPoleAitkens  | 3365 km away | 2115 km from edge 


In [11]:
distribution(Chang_e=5)

Chang'e 5 Landing Site
Sample Location: 43.1 N , -51.8 E


100%|███████████████████████████████████████████| 19/19 [02:09<00:00,  6.83s/it]


Stacked Height at Chang'e 5 Landing Site
0.05295 km of ejecta from all basins
----------------------------------------------------------------------------
0.05225 km (#1) | from Imbrium       | 0918 km away | 0345 km from edge 
0.00012 km (#3) | from Lorentz       | 1083 km away | 0894 km from edge 
0.00039 km (#2) | from Serenitatis   | 1732 km away | 1395 km from edge 
0.00005 km (#6) | from Humorum       | 2094 km away | 1884 km from edge 
0.00008 km (#4) | from Hertzsprung   | 2428 km away | 2143 km from edge 
0.00007 km (#5) | from Crisium       | 2835 km away | 2557 km from edge 
